In [1]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [3]:
df = pd.merge(listings, reviews, on='listing_id')

In [41]:
df.describe();

In [40]:
df.head();

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [62]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['comments'][1:1000]))

In [64]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'best', 'thing', 'about', 'this', 'place', 'is', 'the', 'location', 'you', 're', 'right', 'in', 'the', 'middle', 'of', 'clinton', 'hill', 'with', 'easy', 'access', 'to', 'restaurants', 'on', 'franklin', 'avenue', 'and', 'in', 'fort', 'greene', 'you', 're', 'also', 'close', 'to', 'the', 'and', 'trains', 'so', 'transportation', 'is', 'easy', 'the', 'space', 'is', 'clean', 'and', 'neat', 'if', 'spare', 'it', 'really', 'is', 'much', 'more', 'of', 'one', 'floor', 'apartment', 'than', 'private', 'room', 'as', 'you', 'will', 'only', 'have', 'interactions', 'with', 'the', 'family', 'on', 'the', 'shared', 'stairs', 'and', 'stoop', 'there', 'is', 'however', 'no', 'door', 'at', 'the', 'top', 'of', 'the', 'stairs', 'so', 'sounds', 'travel', 'from', 'the', 'rest', 'of', 'the', 'house', 'lisa_roxanne', 'was', 'kind', 'and', 'accommodating', 'and', 'the', 'children', 'are', 'adorable']


In [65]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [66]:
a =df.comments[1:100].to_string()
len(a)
a

"1     The best thing about this place is the locatio...\n2     LisaRoxanne was friendly and helpful. The room...\n3     LisaRoxanne is very nice and communicative. Th...\n4     Lisa was a very pleasant and nice host that ac...\n5     Lisa était sympa, toujours prête à aider. L'ét...\n6     The reservation was canceled 5 days before arr...\n7     We were greeted by Lisa's husband at the door....\n8     Lisa and her family were warm and welcoming ho...\n9     The place was great. Lisaroxanne was a great h...\n10    Bien que la chambre soit défraichi, nous avons...\n11    LisaRoxanne met us out on the sidewalk, and wa...\n12    She is so wonder full. We got friendly and hom...\n13    LisaRoxanne and her family are some of the mos...\n14    Lisa was great! Communication prior to the sta...\n15    Han sido unos días muy agradables. La casa est...\n16    LisaRoxanne is very kind and accommodative hos...\n17    Nice apartment, good location. Very near to Su...\n18    We had a very good time 

In [69]:
type(data_words)

list

In [70]:
type(output)

list

In [77]:
from nltk.util import ngrams

s = df.comments[1:1000].to_string().lower()
s = s.replace('\n', '')
s = re.sub(r'[1-9]','', s)
s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)

# Remove blank comments
tokens = [token for token in s.split(" ") if token != ""]
tokens = remove_stopwords(tokens)
print('type(tokens): ', type(tokens))

output = list(ngrams(tokens, 2))
print('type(output): ', type(output))
# print(output, 'output')

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
print('type(data_words_nostops): ', type(data_words_nostops))

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
print('type(data_words_bigrams): ', type(data_words_bigrams))
# print(data_words_bigrams)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(str(output), allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

type(tokens):  <class 'list'>
type(output):  <class 'list'>
type(data_words_nostops):  <class 'list'>
type(data_words_bigrams):  <class 'list'>
[[]]


In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1)]]


In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [14]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.454*"stay" + 0.341*"great" + 0.188*"location" + 0.000*"family" + '
  '0.000*"apartment" + 0.000*"wonderful" + 0.000*"perfect" + 0.000*"really" + '
  '0.000*"enjoy" + 0.000*"time"'),
 (1,
  '0.148*"nous" + 0.001*"avon" + 0.001*"be" + 0.001*"journ" + 0.001*"pass" + '
  '0.001*"e" + 0.001*"longer" + 0.001*"long" + 0.001*"lixaroxanne" + '
  '0.001*"pann"'),
 (2,
  '0.803*"good" + 0.000*"eat" + 0.000*"price" + 0.000*"safe" + 0.000*"review" '
  '+ 0.000*"apa" + 0.000*"know" + 0.000*"around" + 0.000*"look" + '
  '0.000*"apartment"'),
 (3,
  '0.001*"davv" + 0.001*"situato" + 0.001*"old" + 0.001*"image" + '
  '0.001*"situated" + 0.001*"situate" + 0.001*"posizione" + 0.001*"explore" + '
  '0.001*"ligge" + 0.001*"ligging"'),
 (4,
  '0.001*"davv" + 0.001*"situato" + 0.001*"old" + 0.001*"image" + '
  '0.001*"situated" + 0.001*"situate" + 0.001*"posizione" + 0.001*"explore" + '
  '0.001*"ligge" + 0.001*"ligging"'),
 (5,
  '0.938*"place" + 0.000*"family" + 0.000*"total" + 0.000*"miss" + '
 

In [15]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -19.848756660517264

Coherence Score:  0.6562155469743248


In [16]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/tristandealwis/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                        x                   y  topics  cluster       Freq
topic                                                                    
0      0.426040+0.000000j -0.342410+0.000000j       1        1  45.617077
15     0.246893+0.000000j  0.155809+0.000000j       2        1  11.646259
5      0.281403+0.000000j  0.269625+0.000000j       3        1  11.494942
17     0.191926+0.000000j  0.072534+0.000000j       4        1   8.850121
11     0.094545+0.000000j  0.011447+0.000000j       5        1   3.759474
2      0.045170+0.000000j -0.003834+0.000000j       6        1   3.731988
19     0.070108+0.000000j  0.003076+0.000000j       7        1   3.468595
9     -0.049276+0.000000j -0.019566+0.000000j       8        1   2.700602
13    -0.024301+0.000000j -0.017196+0.000000j       9        1   2.596212
10    -0.118270+0.000000j -0.012732+0.000000j      10        1   0.829091
1     -0.120298+0.000000j -0.014528+0.000000j      11        1   0.767534
6     -